In [1]:
import pandas as pd
import cptac
import numpy as np
import requests
import json
import deva
import binarization_functions_1 as bf

In [2]:
ov = cptac.Ovarian()
clinical = ov.get_clinical()
proteomics = ov.get_proteomics()

In [3]:
follow_up = pd.read_excel('../Follow_Up_Data/Ovary_One_Year_Clinical_Data_20160927.xls')

In [10]:
for col in follow_up.columns:
    if len(follow_up[col].value_counts()) <= 1:
        follow_up = follow_up.drop(col, axis=1)

cols_to_drop = ['Short Title', 'Event', 'Modified Time', 
                'Modified By', 'Status', 'CRF Name', 
                'Was a Review of the Initial Pathological Findings Done?', 
                'Was the Pathology Review consistent with the Diagnosis?', 
                'Adjuvant (Post-Operative) Radiation Therapy', 
                'Adjuvant (Post-Operative) Immunotherapy', 
                'Adjuvant (Post-Operative) Hormone Therapy', 
                'Anatomic Site of New Tumor Event', 
                'Other Site of New Tumor Event or Lymph Node Location', 
                'Method Of Diagnosis of New Tumor Event', 
                'Date of Additional Surgery for New Tumor Event', 
                'Additional Hormone Therapy Treatment of New Tumor Event', 
                'Additional Targeted Molecular Therapy Treatment of New Tumor Event', 
                'What Type of Malignancy was This?', 'Primary Site of Disease', 
                'Laterality of the Disease', 'Did the patient have surgery for this malignancy?',
                'Number of Days from Date of Initial Pathologic Diagnosis of the Tumor Submitted for CPTAC to the Date of Initial Diagnosis of Other Malignancy', 
                'Did the patient receive pharmaceutical therapy for this malignancy?', 
                'Was the patient staged using FIGO?', ' Was the patient staged using AJCC?']

follow_up = follow_up.drop(cols_to_drop, axis=1).copy()

things_to_replace = ['Not Reported/ Unknown', 'Reported/ Unknown', 
                     'Not Applicable', 'na', 'unknown', 'Not Performed', 
                     'Unknown tumor status', 'Unknown']

for col in follow_up.columns:
    follow_up[col] = follow_up[col].replace(things_to_replace, np.nan)

#This may not be wise to binarize for survival analysis. We shall see
#for col in follow_up.columns:
#    if np.issubdtype(follow_up[col].dtype, np.number):
#        mean = follow_up[col].mean()
#        follow_up[col]= bf.binarizeCutOff(follow_up, col, mean,
#                                          "Above_Mean("+str(round(mean, 2))+")",
#                                          "Below_Mean("+str(round(mean, 2))+")")

chemo_to_replace = ['Chemotherapy', 'Chemo', 'chemotherapy']
phase_to_replace = ['Phase I', 'Phase 1', 'Pahse I', 'With other Phase I drug']
inhibitors_to_replace = ['PARP Inhibitor', 'aromatase inhibitor']
other_to_replace = ['alkylating agent', 'plant alkaloid', 
                    'monoclonal antibody', 'GOG9923']

col = 'Clinical Trial Drug Classification'

follow_up[col] = follow_up[col].replace(chemo_to_replace, 'Chemotherapy')
follow_up[col] = follow_up[col].replace(phase_to_replace, 'Phase I')
follow_up[col] = follow_up[col].replace(inhibitors_to_replace, 'Inhibitors')
follow_up[col] = follow_up[col].replace(other_to_replace, 'Other')

In [12]:
follow_up = follow_up.rename({'PPID': 'Patient_ID'}, axis='columns')

In [14]:
patient_data = pd.merge(clinical, follow_up, on = 'Patient_ID')

In [15]:
patient_data.columns

Index(['Patient_ID', 'Sample_Tumor_Normal', 'Participant_Procurement_Age',
       'Participant_Gender', 'Participant_Race', 'Participant_Ethnicity',
       'Participant_Jewish_Heritage', 'Participant_History_Malignancy',
       'Participant_History_Chemotherapy',
       'Participant_History_Neo-adjuvant_Treatment',
       ...
       'Additional Chemotherapy Treatment of New Tumor Event',
       'Additional Immunotherapy Treatment of New Tumor Event',
       'Was Patient Treated on a Clinical Trial?',
       'Drug Name (Brand or Generic)', 'Clinical Trial Drug Classification',
       'Pharmaceutical Type',
       'Number of Days from Date of Initial Pathologic Diagnosis to Date of Therapy Start',
       'Therapy Ongoing',
       'Number of Days from Date of Initial Pathologic Diagnosis to Date of Therapy End',
       'Measure of Best Response of Pharmaceutical Treatment'],
      dtype='object', length=107)